In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

--2025-05-17 19:46:26--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.230.244.24, 54.230.244.226, 54.230.244.197, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.230.244.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet’

yellow_tripdata_202 100%[===================>]  45.46M   274MB/s    in 0.2s    

2025-05-17 19:46:26 (274 MB/s) - ‘yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]

--2025-05-17 19:46:26--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 54.230.244.24, 54.230.244.226, 54.230.244.197, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|54.230.244.24|:443... conn

In [ ]:
df_jan = pd.read_parquet('yellow_tripdata_2023-01.parquet')
df_feb = pd.read_parquet('yellow_tripdata_2023-02.parquet')
print(f"number of columns in df_jan: {len(df_jan.columns)}")
print(f"number of columns in df_feb: {len(df_feb.columns)}")

number of columns in df_jan: 19
number of columns in df_feb: 19


In [ ]:
df_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [ ]:
df_jan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [ ]:
df_jan['duration'] = df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']
df_jan['duration'] = df_jan['duration'].dt.total_seconds() / 60
df_jan['duration'].describe()

,duration
count,3.066766e+06
mean,1.566900e+01
std,4.259435e+01
min,-2.920000e+01
25%,7.116667e+00
50%,1.151667e+01
75%,1.830000e+01
max,1.002918e+04


In [ ]:
std_duration = df_jan['duration'].std()
std_duration

42.594351241920904

In [ ]:
initial_count = len(df_jan)
df_jan_filtered = df_jan[(df_jan['duration'] >= 1) & (df_jan['duration'] <= 60)]
filtered_count = len(df_jan_filtered)
fraction = filtered_count / initial_count
print(f"The fraction of rows between 1 and 60 minutes is: {fraction:.2f}")
print(f"The number of rows between 1 and 60 minutes is: {filtered_count}")

The fraction of rows between 1 and 60 minutes is: 0.98
The number of rows between 1 and 60 minutes is: 3009173


In [ ]:
from sklearn.feature_extraction import DictVectorizer

df_jan_filtered.loc[:, 'PULocationID'] = df_jan_filtered['PULocationID'].astype(str)
df_jan_filtered.loc[:, 'DOLocationID'] = df_jan_filtered['DOLocationID'].astype(str)

dicts = df_jan_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

vec = DictVectorizer()
X = vec.fit_transform(dicts)
X.shape


(3009173, 515)

In [ ]:
df_jan_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3009173 entries, 0 to 3066765
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           object        
 8   DOLocationID           object        
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee            floa

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

y = df_jan_filtered['duration'].values

lr = LinearRegression()
lr.fit(X, y)
y_pred = lr.predict(X)
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
print(f"The RMSE of the model is: {rmse}")

The RMSE of the model is: 7.649261932106969


In [ ]:
# Test on df_feb
df_feb['duration'] = df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']
df_feb['duration'] = df_feb['duration'].dt.total_seconds() / 60
df_feb_filtered = df_feb[(df_feb['duration'] >= 1) & (df_feb['duration'] <= 60)]
df_feb_filtered.loc[:, 'PULocationID'] = df_feb_filtered['PULocationID'].astype(str)
df_feb_filtered.loc[:, 'DOLocationID'] = df_feb_filtered['DOLocationID'].astype(str)

dicts = df_feb_filtered[['PULocationID', 'DOLocationID']].to_dict(orient='records')

X = vec.transform(dicts)
y = df_feb_filtered['duration'].values

y_pred = lr.predict(X)
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse)
print(f"The RMSE of the model on df_feb is: {rmse}")

<ipython-input-28-bd10e3955aba>:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['142' '132' '161' ... '158' '79' '161']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_feb_filtered.loc[:, 'PULocationID'] = df_feb_filtered['PULocationID'].astype(str)
<ipython-input-28-bd10e3955aba>:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['163' '26' '145' ... '143' '162' '140']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df_feb_filtered.loc[:, 'DOLocationID'] = df_feb_filtered['DOLocationID'].astype(str)


The RMSE of the model on df_feb is: 7.811818743246608
